In [ ]:
config_file = "upsnet/experiments/upsnet_resnet50_ade20k_4gpu.yaml"
if False:
    with open(config_file) as f:
        print(f.read())
final_output_path = 'results'

In [ ]:
import os
import sys
import logging
import pprint
import time
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.utils.data
import torch.backends.cudnn as cudnn
import cv2
from PIL import Image
from upsnet.config.config import config, update_config
from lib.utils.logging import create_logger
from lib.utils.timer import Timer

import warnings
warnings.filterwarnings("ignore")

update_config(config_file)
config.output_path = final_output_path
logger, final_output_path = create_logger(config.output_path, config_file, config.dataset.test_image_set)

from upsnet.dataset import *
from upsnet.models import *
from upsnet.bbox.bbox_transform import bbox_transform, clip_boxes, expand_boxes
from lib.utils.callback import Speedometer
from lib.utils.data_parallel import DataParallel
from pycocotools.mask import encode as mask_encode

Load Tony's trained segmentation model

In [ ]:
weights_file = "/data/vision/torralba/scratch/tonypeng/upsnet/output/ade20k/upsnet_resnet50_ade20k_4gpu/frames_training_places_training/upsnet_resnet_50_ade20k_387000.state.pth"

gpus = [int(_) for _ in config.gpus.split(',')]
test_model = eval(config.symbol)().cuda(device=gpus[0])
test_model.load_state_dict(torch.load(weights_file))


Load a small test dataset

In [ ]:
test_dataset = eval(config.dataset.dataset)(image_sets=config.dataset.test_image_set.split('+'), flip=False,
                                            result_path=final_output_path, phase='test')
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.test.batch_size, shuffle=False,
                                         num_workers=0, drop_last=False, pin_memory=False, collate_fn=test_dataset.collate)
with open('seg-classes.txt') as f:
    segclasses = [line.split(':')[1].split(',')[0].strip() for line in f.readlines()]
    
print(len(test_dataset), 'images')

Execute segmentation on one image

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from viz import show, segviz
import PIL

image_number = 500

data, label, _ = test_dataset.collate([test_dataset[image_number]])
if label is not None:
    data['roidb'] = label['roidb']
for k, v in data.items():
    data[k] = v.pin_memory().to('cuda', non_blocking=True) if torch.is_tensor(v) else v
    
    
results = test_model(data)

imdata = (data['data'][0].cpu().permute(1,2,0) + torch.tensor([[[103.0, 116.0, 123.0]]])).clamp(0,255).byte()
segdata = results['fcn_outputs'][0]

def smaller(pil_image):
    target_height = 200
    target_width = pil_image.width * target_height // pil_image.height
    return pil_image.resize((target_width, target_height))

show([[smaller(PIL.Image.fromarray(imdata.numpy()))],
      [smaller(segviz.seg_as_image(segdata))],
      [segviz.segment_key(segdata, classes=segclasses)]])

In [ ]:
results.keys()

In [ ]:
results

In [ ]:
segdata.min()

In [ ]:
type(test_dataset)

In [ ]:
print([data['data'][0][i].cpu().min() for i in range(3)])